# SPR 2026 - Word2Vec NILC (Pré-treinado)

**NILC Word Embeddings**

Embeddings pré-treinados em um corpus massivo de português brasileiro pelo NILC-USP.
Contém mais conhecimento linguístico do que treinar do zero.

---
**CONFIGURAÇÃO:**
- Requer adicionar dataset NILC embeddings no Kaggle
- Dataset: [NILC Word Embeddings](http://www.nilc.icmc.usp.br/nilc/index.php/repositorio-de-word-embeddings-do-nilc)
- Arquivo: `cbow_s100.txt` ou similar

**Alternativa:** Se não tiver os embeddings, este notebook treina Word2Vec do zero.
---

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from gensim.models import KeyedVectors, Word2Vec
import os
import re
import warnings
warnings.filterwarnings('ignore')

SEED = 42
EMBEDDING_DIM = 100
DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'
# NILC_PATH = '/kaggle/input/nilc-word-embeddings/cbow_s100.txt'  # Se disponível

np.random.seed(SEED)
print('Bibliotecas carregadas!')

In [ ]:
# Carregar dados
train = pd.read_csv(f'{DATA_DIR}/train.csv')
test = pd.read_csv(f'{DATA_DIR}/test.csv')

print(f'Train: {train.shape}')
print(f'Test: {test.shape}')

In [ ]:
# Preprocessamento
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-záàâãéèêíïóôõöúçñ\s]', ' ', text)
    return text.split()

train['tokens'] = train['report'].apply(preprocess)
test['tokens'] = test['report'].apply(preprocess)

print(f'Exemplo: {train["tokens"].iloc[0][:10]}')

In [ ]:
# Tentar carregar NILC embeddings, senão treinar do zero
USE_PRETRAINED = False

try:
    # Tentar carregar embeddings pré-treinados
    NILC_PATH = '/kaggle/input/nilc-word-embeddings/cbow_s100.txt'
    if os.path.exists(NILC_PATH):
        print('Carregando NILC embeddings...')
        w2v = KeyedVectors.load_word2vec_format(NILC_PATH)
        USE_PRETRAINED = True
        print(f'NILC vocabulário: {len(w2v)} palavras')
except Exception as e:
    print(f'NILC não disponível: {e}')

if not USE_PRETRAINED:
    print('Treinando Word2Vec do zero...')
    all_texts = train['tokens'].tolist() + test['tokens'].tolist()
    
    # Word2Vec com mais epochs para compensar dataset menor
    w2v_model = Word2Vec(
        sentences=all_texts,
        vector_size=EMBEDDING_DIM,
        window=5,
        min_count=1,  # Manter mais palavras
        workers=4,
        epochs=20,  # Mais epochs
        seed=SEED
    )
    w2v = w2v_model.wv
    print(f'Vocabulário: {len(w2v)} palavras')

In [ ]:
# Converter textos em embeddings
def text_to_embedding(tokens, model, dim):
    vectors = [model[w] for w in tokens if w in model]
    if len(vectors) == 0:
        return np.zeros(dim)
    return np.mean(vectors, axis=0)

X_train = np.array([text_to_embedding(t, w2v, EMBEDDING_DIM) for t in train['tokens']])
X_test = np.array([text_to_embedding(t, w2v, EMBEDDING_DIM) for t in test['tokens']])
y_train = train['target'].values

print(f'X_train shape: {X_train.shape}')

In [ ]:
# Treinar modelo
model = LogisticRegression(
    C=1.0,
    max_iter=1000,
    class_weight='balanced',
    random_state=SEED,
    n_jobs=-1
)

model.fit(X_train, y_train)
print('Modelo treinado!')

In [ ]:
# Predições e submissão
predictions = model.predict(X_test)

submission = pd.DataFrame({
    'ID': test['ID'],
    'target': predictions
})

submission.to_csv('submission.csv', index=False)

print('submission.csv criado!')
print(f'Usou embeddings pré-treinados: {USE_PRETRAINED}')
print(submission['target'].value_counts().sort_index())